In [1]:
import sys

# print("sys.path",sys.path)
# sys.path.remove('/data/dengjunli/DJL_workspace/DINet')
# sys.path.append('/data/dengjunli/DJL_workspace/DINet-2大数据集训练/DINet')
# sys.path.remove('/data/dengjunli/talking_lip-放在这里只是看的别跑')
# sys.path.remove('/data/dengjunli/talking_lip-放在这里只是看的别跑/wav2lip')
# sys.path.remove('/data/dengjunli/talking_lip-放在这里只是看的别跑/codeformer')
# print("sys.path",sys.path)
sys.path.append('../')

from utils.deep_speech import DeepSpeech
from utils.data_processing import load_landmark_openface,compute_crop_radius
from config.config import DINetInferenceOptions
# from models.DINet import DINet
from models.LawDNet_new import LawDNet
import os
from tensor_processing import SmoothSqMask


import numpy as np
import glob
import os
import cv2
import torch
import subprocess
import random
from collections import OrderedDict
from tensor_processing import *
import torch.nn.functional as F

2023-09-25 17:11:18.408267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 17:11:19.239620: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 初始化必要的模块
# 先固定opt参数
source_video_path = "./asserts/examples/bilibili_dataset_25fps_FFoutputzynd1iox_part1_corrected.mp4"
source_openface_landmark_path = "./asserts/examples/bilibili_dataset_25fps_FFoutputzynd1iox_part1_corrected.csv"
# driving_audio_path = "./asserts/examples/RD_Radio14_000_corrected.wav"

# source_video_path = "./asserts/examples/test1.mp4"
# source_openface_landmark_path = "./asserts/examples/test1.csv"
driving_audio_path = "./asserts/examples/bilibili_dataset_25fps_FFoutput0e3w13uh_part8_corrected.wav"

# 实验1
pretrained_clip_DINet_path = "./asserts/training_model_weight/clip_training_256/netG_model_epoch_4.pth"

# dengjunli复现作者的
# pretrained_clip_DINet_path = "./asserts/training_model_weight-复现作者原模型/clip_training_256/netG_model_epoch_100.pth"

# 作者自己的
# pretrained_clip_DINet_path = "./asserts/clip_training_DINet_256mouth.pth"

deepspeech_model_path = "./asserts/output_graph.pb"
mouth_region_size = 256
res_video_dir = './asserts/inference_result'
source_channel = 3
ref_channel = 15
audio_channel=29


In [3]:
def extract_frames_from_video(video_path,save_dir):
    videoCapture = cv2.VideoCapture(video_path)
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    if int(fps) != 25:
        print('warning: the input video is not 25 fps, it would be better to trans it to 25 fps!')
    frames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frame_width = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)
    for i in range(int(frames)):
        ret, frame = videoCapture.read()
        result_path = os.path.join(save_dir, str(i).zfill(6) + '.jpg')
        cv2.imwrite(result_path, frame)
    return (int(frame_width),int(frame_height))

In [4]:
# 设置 PyTorch 只使用第一张 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("已设置 PyTorch 只使用第一张 GPU 卡")
mask_gaussian = Gaussian_bluring(radius=30,sigma=10,padding='same').cuda()

# load config
# opt = DINetInferenceOptions().parse_args()
if not os.path.exists(source_video_path):
    raise ('wrong video path : {}'.format(source_video_path))
############################################## extract frames from source video ##############################################
print('extracting frames from video: {}'.format(source_video_path))
video_frame_dir = source_video_path.replace('.mp4', '')
if not os.path.exists(video_frame_dir):
    os.mkdir(video_frame_dir)
video_size = extract_frames_from_video(source_video_path,video_frame_dir)
############################################## extract deep speech feature ##############################################
print('extracting deepspeech feature from : {}'.format(driving_audio_path))
if not os.path.exists(deepspeech_model_path):
    raise ('pls download pretrained model of deepspeech')
DSModel = DeepSpeech(deepspeech_model_path)
if not os.path.exists(driving_audio_path):
    raise ('wrong audio path :{}'.format(driving_audio_path))
ds_feature = DSModel.compute_audio_feature(driving_audio_path)
res_frame_length = ds_feature.shape[0]
ds_feature_padding = np.pad(ds_feature, ((2, 2), (0, 0)), mode='edge')
############################################## load facial landmark ##############################################
print('loading facial landmarks from : {}'.format(source_openface_landmark_path))
video_frame_path_list = glob.glob(os.path.join(video_frame_dir, '*.jpg'))
if not os.path.exists(source_openface_landmark_path):
    raise ('wrong facial landmark path :{}'.format(source_openface_landmark_path))
video_landmark_data = load_landmark_openface(source_openface_landmark_path,len(video_frame_path_list)).astype(np.int64)
############################################## align frame with driving audio ##############################################
print('aligning frames with driving audio')

已设置 PyTorch 只使用第一张 GPU 卡


extracting frames from video: ./asserts/examples/bilibili_dataset_25fps_FFoutputzynd1iox_part1_corrected.mp4
extracting deepspeech feature from : ./asserts/examples/bilibili_dataset_25fps_FFoutput0e3w13uh_part8_corrected.wav


/root/autodl-tmp/DINet-gitee/DINet-update/utils/deep_speech.py:74: UserWarning: Audio has multiple channels, the first channel is used
  warnings.warn(
2023-09-25 17:12:25.679231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17616 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:c1:00.0, compute capability: 8.6


看这个MFCC feature 是否一样: 
看这个input vector是否一样


2023-09-25 17:12:26.482140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-09-25 17:12:55.070127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


loading facial landmarks from : ./asserts/examples/bilibili_dataset_25fps_FFoutputzynd1iox_part1_corrected.csv
aligning frames with driving audio


############################################## align frame with driving audio ##############################################

In [5]:
print(ds_feature.shape)
print(ds_feature_padding.shape)
print(type(ds_feature))

(1499, 29)
(1503, 29)
<class 'numpy.ndarray'>


In [6]:
video_frame_path_list.sort()
video_frame_path_list_cycle = video_frame_path_list + video_frame_path_list[::-1]
video_landmark_data_cycle = np.concatenate([video_landmark_data, np.flip(video_landmark_data, 0)], 0)
video_frame_path_list_cycle_length = len(video_frame_path_list_cycle)
if video_frame_path_list_cycle_length >= res_frame_length:
    res_video_frame_path_list = video_frame_path_list_cycle[:res_frame_length]
    res_video_landmark_data = video_landmark_data_cycle[:res_frame_length, :, :]
else:
    divisor = res_frame_length // video_frame_path_list_cycle_length
    remainder = res_frame_length % video_frame_path_list_cycle_length
    res_video_frame_path_list = video_frame_path_list_cycle * divisor + video_frame_path_list_cycle[:remainder]
    res_video_landmark_data = np.concatenate([video_landmark_data_cycle]* divisor + [video_landmark_data_cycle[:remainder, :, :]],0)
res_video_frame_path_list_pad = [video_frame_path_list_cycle[0]] * 2 \
                                + res_video_frame_path_list \
                                + [video_frame_path_list_cycle[-1]] * 2
res_video_landmark_data_pad = np.pad(res_video_landmark_data, ((2, 2), (0, 0), (0, 0)), mode='edge')
assert ds_feature_padding.shape[0] == len(res_video_frame_path_list_pad) == res_video_landmark_data_pad.shape[0]
pad_length = ds_feature_padding.shape[0]

########## randomly select 5 reference images ###########

In [7]:
print('selecting five reference images')
ref_img_list = []
resize_w = int(mouth_region_size + mouth_region_size // 4)
resize_h = int((mouth_region_size // 2) * 3 + mouth_region_size // 8)
ref_index_list = random.sample(range(5, len(res_video_frame_path_list_pad) - 2), 5)
for ref_index in ref_index_list:
    crop_flag,crop_radius = compute_crop_radius(video_size,res_video_landmark_data_pad[ref_index - 5:ref_index, :, :])
    if not crop_flag:
        raise ('our method can not handle videos with large change of facial size!!')
    crop_radius_1_4 = crop_radius // 4
    ref_img = cv2.imread(res_video_frame_path_list_pad[ref_index- 3])[:, :, ::-1]
    ref_landmark = res_video_landmark_data_pad[ref_index - 3, :, :]
    ref_img_crop = ref_img[
                ref_landmark[29, 1] - crop_radius:ref_landmark[29, 1] + crop_radius * 2 + crop_radius_1_4,
                ref_landmark[33, 0] - crop_radius - crop_radius_1_4:ref_landmark[33, 0] + crop_radius +crop_radius_1_4,
                :]
    ref_img_crop = cv2.resize(ref_img_crop,(resize_w,resize_h))
    ref_img_crop = ref_img_crop / 255.0
    ref_img_list.append(ref_img_crop)
ref_video_frame = np.concatenate(ref_img_list, 2)
ref_img_tensor = torch.from_numpy(ref_video_frame).permute(2, 0, 1).unsqueeze(0).float().cuda()


selecting five reference images


##### load pretrained model weight ########

In [8]:
print('loading pretrained model from: {}'.format(pretrained_clip_DINet_path))
# model = DINet(source_channel, ref_channel, audio_channel).cuda()
model = LawDNet(source_channel, ref_channel, audio_channel).cuda()

if not os.path.exists(pretrained_clip_DINet_path):
    raise ('wrong path of pretrained model weight: {}'.format(pretrained_clip_DINet_path))

state_dict = torch.load(pretrained_clip_DINet_path)['state_dict']['net_g']

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]  # remove module.
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()

loading pretrained model from: ./asserts/training_model_weight/clip_training_256/netG_model_epoch_4.pth


LawDNet(
  (source_in_conv): Sequential(
    (0): SameBlock2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU()
    )
    (1): DownBlock2d(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (relu): ReLU()
    )
    (2): DownBlock2d(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (relu): ReLU()
    )
  )
  (ref_in_conv): Sequential(
    (0): SameBlock2d(
      (conv): Conv2d(15, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU()
    )
    (1): DownBlock2d(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (relu): ReLU()
    )
    (2): DownBlock2d(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (relu): ReLU()
    )
  )
  (trans_conv): Sequential(
    (0): SameBlock2d(
      (conv): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu

In [9]:
radius = mouth_region_size // 2
radius_1_4 = radius // 4

######### inference frame by frame ##########

In [17]:
if not os.path.exists(res_video_dir):
    os.mkdir(res_video_dir)
res_video_path = os.path.join(res_video_dir,os.path.basename(source_video_path)[:-4] + '_facial_dubbing.mp4')
if os.path.exists(res_video_path):
    os.remove(res_video_path)
res_face_path = res_video_path.replace('_facial_dubbing.mp4', '_synthetic_face.mp4')
print("res_video_path: ", res_video_path)

if os.path.exists(res_face_path):
    os.remove(res_face_path)
videowriter = cv2.VideoWriter(res_video_path, cv2.VideoWriter_fourcc(*'XVID'), 25, video_size)
videowriter_face = cv2.VideoWriter(res_face_path, cv2.VideoWriter_fourcc(*'XVID'), 25, (resize_w, resize_h))

# 创建一个空的张量，用于存储所有的 deepspeech_tensor
deepspeech_tensor_all = torch.empty((ds_feature.shape[0], 29, 5))
i = 0
smooth_sqmask = SmoothSqMask().cuda()

for clip_end_index in range(5, pad_length, 1):
    print('synthesizing {}/{} frame'.format(clip_end_index - 5, pad_length - 5))
    crop_flag, crop_radius = compute_crop_radius(video_size,res_video_landmark_data_pad[clip_end_index - 5:clip_end_index, :, :],random_scale = 1.05)
    if not crop_flag:
        raise ('our method can not handle videos with large change of facial size!!')
    crop_radius_1_4 = crop_radius // 4
    frame_data = cv2.imread(res_video_frame_path_list_pad[clip_end_index - 3])[:, :, ::-1]
    frame_landmark = res_video_landmark_data_pad[clip_end_index - 3, :, :]
    crop_frame_data = frame_data[
                        frame_landmark[29, 1] - crop_radius:frame_landmark[29, 1] + crop_radius * 2 + crop_radius_1_4,
                        frame_landmark[33, 0] - crop_radius - crop_radius_1_4:frame_landmark[33, 0] + crop_radius +crop_radius_1_4,
                        :]
    crop_frame_h,crop_frame_w = crop_frame_data.shape[0],crop_frame_data.shape[1]
    crop_frame_data = cv2.resize(crop_frame_data, (resize_w,resize_h))  # [32:224, 32:224, :]
    crop_frame_data = crop_frame_data / 255.0
    # crop_frame_data[mouth_region_size//2:mouth_region_size//2 + mouth_region_size,
    #                 mouth_region_size//8:mouth_region_size//8 + mouth_region_size, :] = 0

    crop_frame_tensor = torch.from_numpy(crop_frame_data).float().cuda().permute(2, 0, 1).unsqueeze(0)
    deepspeech_tensor = torch.from_numpy(ds_feature_padding[clip_end_index - 5:clip_end_index, :]).permute(1, 0).unsqueeze(0).float().cuda()

    ##################### yihao softmask #####################
    # source_clip_alpha = torch.zeros_like(crop_frame_tensor[:,0:1,...])
    # source_clip_alpha[:,:,radius:radius+mouth_region_size,
    #                         radius_1_4:radius_1_4+mouth_region_size] = 1
    # source_clip_alpha = mask_gaussian(source_clip_alpha)  # radius=30,sigma=10
    # crop_frame_tensor_soft_mask = crop_frame_tensor*(1.0-source_clip_alpha)
    ##################### yihao softmask #####################


    ###################################################
    feed_img = torch.tensor(frame_data.copy(),device=device).float().permute(2,0,1).unsqueeze(0)
    landmarks_tensor = torch.from_numpy(frame_landmark).float().to(device).unsqueeze(0)
    facealigner = FaceAlign(device=device,ratio=1.6)
    face_align_img, lmrks_align, face_align_matrix = facealigner(feed_img, landmarks_tensor)
    face_align_img_resize = F.interpolate(face_align_img, size=(resize_h,resize_w), mode='bilinear', align_corners=True)


    face_align_img_resize = face_align_img_resize / 255.0
    #######

    ###################### 邓 test  #####################
    
    # one = torch.load("./测试用/autodl3090拿过来的模型/邓-source_clip_mask.pt")
    # two = torch.load("./测试用/autodl3090拿过来的模型/邓-reference_clip.pt")
    # three = torch.load("./测试用/autodl3090拿过来的模型/邓-deep_speech_clip.pt")
    # pre_frame_1 = model(one, two, three)
    # pre_frame_1 = pre_frame_1[0].permute(1, 2, 0).detach().cpu().numpy() * 255
    # cv2.imwrite('./邓-train-实验10-pre_frame-output.jpg', pre_frame_1[:, :, ::-1].copy().astype(np.uint8))

    crop_frame_tensor_soft_mask=smooth_sqmask(face_align_img_resize)

    ###################### 邓 test  #####################

    
    with torch.no_grad():
        crop_frame_tensor_soft_mask_np = crop_frame_tensor_soft_mask.squeeze(0).permute(1, 2, 0).detach().cpu().numpy() * 255
        # cv2.imwrite('./邓-inference-实验10-crop_frame_tensor_soft_mask.jpg', crop_frame_tensor_soft_mask_np[:, :, ::-1].copy().astype(np.uint8))
        # pre_frame = model(crop_frame_tensor_soft_mask, ref_img_tensor, deepspeech_tensor)
        deepspeech_tensor_all[i] = deepspeech_tensor.squeeze()
#         # i = i + 1
#         pre_frame = pre_frame.squeeze(0).permute(1, 2, 0).detach().cpu().numpy() * 255
#         cv2.imwrite('./邓-inference-实验10-pre_frame.jpg', pre_frame[:, :, ::-1].copy().astype(np.uint8))
#     videowriter_face.write(pre_frame[:, :, ::-1].astype(np.uint8))
#     pre_frame_resize = cv2.resize(pre_frame.astype(float), (crop_frame_w,crop_frame_h))
#     frame_data[
#     frame_landmark[29, 1] - crop_radius:
#     frame_landmark[29, 1] + crop_radius * 2,
#     frame_landmark[33, 0] - crop_radius - crop_radius_1_4:
#     frame_landmark[33, 0] + crop_radius + crop_radius_1_4,
#     :] = pre_frame_resize[:crop_radius * 3,:,:]
#     videowriter.write(frame_data[:, :, ::-1])
# videowriter.release()
# videowriter_face.release()
# video_add_audio_path = res_video_path.replace('.mp4', '_add_audio.mp4')
# if os.path.exists(video_add_audio_path):
#     os.remove(video_add_audio_path)
# cmd = 'ffmpeg -i {} -i {} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 {}'.format(
#     res_video_path,
#     driving_audio_path,
#     video_add_audio_path)
# print("save video with audio: {}".format(video_add_audio_path))
# print("save path: {}".format(res_video_dir))
# subprocess.call(cmd, shell=True)


res_video_path:  ./asserts/inference_result/bilibili_dataset_25fps_FFoutputzynd1iox_part1_corrected_facial_dubbing.mp4
synthesizing 0/1498 frame
synthesizing 1/1498 frame
synthesizing 2/1498 frame
synthesizing 3/1498 frame


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


synthesizing 4/1498 frame
synthesizing 5/1498 frame
synthesizing 6/1498 frame
synthesizing 7/1498 frame
synthesizing 8/1498 frame
synthesizing 9/1498 frame
synthesizing 10/1498 frame
synthesizing 11/1498 frame
synthesizing 12/1498 frame
synthesizing 13/1498 frame
synthesizing 14/1498 frame
synthesizing 15/1498 frame
synthesizing 16/1498 frame
synthesizing 17/1498 frame
synthesizing 18/1498 frame
synthesizing 19/1498 frame
synthesizing 20/1498 frame
synthesizing 21/1498 frame
synthesizing 22/1498 frame
synthesizing 23/1498 frame
synthesizing 24/1498 frame
synthesizing 25/1498 frame
synthesizing 26/1498 frame
synthesizing 27/1498 frame
synthesizing 28/1498 frame
synthesizing 29/1498 frame
synthesizing 30/1498 frame
synthesizing 31/1498 frame
synthesizing 32/1498 frame
synthesizing 33/1498 frame
synthesizing 34/1498 frame
synthesizing 35/1498 frame
synthesizing 36/1498 frame
synthesizing 37/1498 frame
synthesizing 38/1498 frame
synthesizing 39/1498 frame
synthesizing 40/1498 frame
synthes

print(pre_frame_1.shape)
print(pre_frame.shape)

In [18]:
torch.save(deepspeech_tensor_all,"./中文男-deepspeech_tensor_all.pt")

In [19]:
# print(frame_data.shape)
# print(type(frame_data))

In [20]:
# print(one.shape)
# print(crop_frame_tensor_soft_mask.shape)
# print(three.shape)


In [21]:
# print(two.shape)
# print(ref_img_tensor.shape)
# print(deepspeech_tensor.shape)